In [1]:
import logging
logger = logging.getLogger(__name__)
import escher
import cobra
import cobrakbase
import modelseed_escher
%run ../../scripts/bios_utils.py
import copy
def clone_escher_map(em):
    data = copy.deepcopy(em.escher_map)
    return modelseed_escher.core.EscherMap(data)

cobrakbase 0.2.0


In [3]:
modelseed_local = cobrakbase.modelseed.from_local('/Users/fliu/workspace/jupyter/ModelSEEDDatabase')

load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/reactions.tsv
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/compounds.tsv
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/Structures/Unique_ModelSEED_Structures.txt
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Compound_Aliases.txt
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Reaction_Aliases.txt
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Reaction_ECs.txt


In [77]:
%run convert_utils.py

def get_raw_stoich(db_id, target_db):
    for r in raw_model['reactions']:
        #print(r['id'], r['id'].endswith('@' + target_db))
        if r['id'].endswith('@' + target_db):
            rxn_id, namespace = r['id'].split('@')
            if rxn_id == db_id:
                cstoich = {}
                for m in r['metabolites']:
                    cstoich[(m, '0')] = r['metabolites'][m]
                return cstoich
    return None

def remap_map_reactions(em, bigg_to_seed_rxn, map_compound_remap, get_rxn, to_match_func = lambda x : x, id_func = lambda x : x):
    unmaped_rxn = set()
    map_reaction_remap = {}
    for map_uid in em.escher_graph['reactions']:
        rnode = em.escher_graph['reactions'][map_uid]
        node_id = id_func(rnode['bigg_id'])
        if node_id in bigg_to_seed_rxn:
            for db_id in bigg_to_seed_rxn[node_id]:
                map_stoichiometry = get_stoichiometry(rnode)
                #print(db_id)
                #NEED KEGG/METACYC/BIGG provenance
                rxn_cstoich = get_rxn(db_id)
                
                #print(rxn_cstoich)
                if rxn_cstoich != None:
                    to_match = set(map(lambda x : to_match_func(x[0]), rxn_cstoich.keys()))
                    
                    #print(to_match, map_stoichiometry)
                    mapping, missing, to_match = match2(to_match, map_stoichiometry, map_compound_remap)
                    missing -= ALLOW_MISSING
                    #print(missing)
                    if len(missing) == 0:
                        #print(map_uid, node_id, db_id, map_stoichiometry, rxn.cstoichiometry)
                        #print(mapping, missing, to_match)
                        if not node_id in map_reaction_remap:
                            map_reaction_remap[rnode['bigg_id']] = set()
                        map_reaction_remap[rnode['bigg_id']].add(db_id)
                        for s_id in rnode['segments']:
                            s = rnode['segments'][s_id]
                            #print(s_id, s)
                            break
                    else:
                        print(node_id, db_id, missing)
                        unmaped_rxn.add(node_id)
        else:
            logger.debug(node_id)
            unmaped_rxn.add(node_id)
    return map_reaction_remap, unmaped_rxn

#map_reaction_remap, unmaped_rxn = remap_map_reactions(em, rxn_bigg_to_seed, map_compound_remap, 
#                                                      lambda x : modelseed_local.get_seed_reaction(x.split('@')[0]).cstoichiometry,
#                                                      lambda x : x + '@SEED', lambda x : x[:-2])

In [78]:
base = '/Users/fliu/Library/Caches/escher/1-0-0/5/maps/'
map_file = base + '/Saccharomyces cerevisiae/iMM904_SEED.Central carbon metabolism Large.json'
#map_file = base + '/Escherichia coli/iJO1366.Fatty acid beta-oxidation.json'
#map_file = base + '/Escherichia coli/iJO1366.Fatty acid biosynthesis (saturated).json'
em_bigg = modelseed_escher.core.EscherMap(read_json(map_file))
cpd_remap = {
    '2pg_c' : 'swaped!'
}
rxn_remap = {}
res = em_bigg.add_uid_to_reaction_metabolites()
#add_uid_to_reaction_metabolites(em_bigg)
#pass
#em.swap_ids(cpd_remap, rxn_remap)

In [79]:
em_bigg.display_in_notebook()

In [80]:
cpd_bigg_to_seed = dict(map(lambda x : (x, set([x + '@SEED'])), modelseed_local.compounds))
rxn_bigg_to_seed = dict(map(lambda x : (x, set([x + '@SEED'])), modelseed_local.reactions))


In [81]:
ALLOW_MISSING.add('cpd00067@SEED')
em = clone_escher_map(em_bigg)
map_compound_remap, unmaped_cpd, node_uid_cmp = remap_map_compounds(em, cpd_bigg_to_seed, 
                                                                    lambda x : (x[:-2], x[-1:]))
map_reaction_remap, unmaped_rxn = remap_map_reactions(em, rxn_bigg_to_seed, map_compound_remap, 
                                                      lambda x : modelseed_local.get_seed_reaction(x.split('@')[0]).cstoichiometry,
                                                      lambda x : x + '@SEED',
                                                      lambda x : x[:-2])

In [82]:
add_compartment(em, node_uid_cmp)
em.delete_reactions(unmaped_rxn)
for k in map_compound_remap:
    cpd_remap[k] = list(map_compound_remap[k])[0]
for k in map_reaction_remap:
    rxn_remap[k] = list(map_reaction_remap[k])[0]
em.swap_ids(cpd_remap, rxn_remap)

em_seed = clone_escher_map(em)

b = escher.Builder(map_json=json.dumps(em.escher_map), model_json=None)
#b.set_highlight_missing(True)
b.display_in_notebook(enable_editing=False)
#b.save_html('../../../data/www/fungi_viz/expanded_seed_central.html')

In [86]:
master_fungi = read_json('../../notebooks/fungi_map/fungi_viz/data/master_model3.json')

In [87]:
logger.setLevel(logging.INFO)
def escher_map_to_model_metabolites(em, model):
    remap = {}
    unmapped = set()
    cmp_to_cpd = {}
    for m in model['metabolites']:
        if m['id'].startswith('cpd'):
            cmp = m['compartment']
            if not cmp in cmp_to_cpd:
                cmp_to_cpd[cmp] = {}
            database_id = m['id'][:8]
            cmp_to_cpd[cmp][database_id] = m['id']
            #print(cmp_to_cpd)
            #break
            
    for m in em.metabolites:
        node_id = m['bigg_id']
        
        if 'compartment' in m:
            cmp = m['compartment'] + '0'
            cpd_id = node_id
            if '@' in cpd_id:
                cpd_id, database = cpd_id.split('@')
            #print(cpd_id)
            logger.debug('%s %s %s', node_id, cmp, cpd_id)
            if cmp in cmp_to_cpd and cpd_id in cmp_to_cpd[cmp]:
                remap[(node_id, m['compartment'])] = cmp_to_cpd[cmp][cpd_id]
            else:
                unmapped.add(node_id)
        else:
            unmapped.add(node_id)
                #print(m, cmp, cpd_id, cmp_to_cpd[cmp][cpd_id])
        #m['compartment'] = 'c0'
        #m['database'] = 'SEED'
        #print(m)
                #break
    return remap, unmapped

def get_cstoich_list(em, rnode):
    cstoich_list = []
    
    for s_uid in rnode['segments']:
        s = rnode['segments'][s_uid]
        from_node_id = rnode['segments'][s_uid]['from_node_id']
        to_node_id = rnode['segments'][s_uid]['to_node_id']
        from_node = em.escher_graph['nodes'][from_node_id]
        to_node = em.escher_graph['nodes'][to_node_id]
        if from_node['node_type'] == 'metabolite':
            cstoich_list.append((from_node['bigg_id'], from_node['compartment']))
        if to_node['node_type'] == 'metabolite':
            cstoich_list.append((to_node['bigg_id'], to_node['compartment']))
            
    return cstoich_list

def escher_map_to_model_reactions(em, model):
    remap = {}
    unmapped = set()
    cmp_to_cpd = {}
    metabolites = {}
    reactions = {}
    for m in model['metabolites']:
        metabolites[m['id']] = m
        if m['id'].startswith('cpd'):
            cmp = m['compartment']
            if not cmp in cmp_to_cpd:
                cmp_to_cpd[cmp] = {}
            database_id = m['id'][:8]
            cmp_to_cpd[cmp][database_id] = m['id']
            #print(cmp_to_cpd)
            #break
    cmp_to_rxn = {}
    for r in model['reactions']:
        if r['id'].startswith('rxn'):
            database_id = r['id'][:8]
            rxn_cmps = set()
            for rxn_m in r['metabolites']:
                cpd = metabolites[rxn_m]
                rxn_cmps.add(cpd['compartment'])
            rxn_cmps = tuple(sorted(list(rxn_cmps)))
            if not rxn_cmps in cmp_to_rxn:
                cmp_to_rxn[rxn_cmps] = {}
            cmp_to_rxn[rxn_cmps][database_id] = r['id']
            
    #print(cmp_to_rxn.keys())
    mapped = set()
    for node_uid in em.escher_graph['reactions']:
        #print(node_uid)
        rnode = em.escher_graph['reactions'][node_uid]
        node_id = rnode['bigg_id']
        cstoich_list = get_cstoich_list(em, rnode)
        rxn_cmps = tuple(sorted(set(map(lambda x : x[1] + '0', cstoich_list))))
        #print(node_uid, node_id, rxn_cmps)
        if rxn_cmps in cmp_to_rxn:
            rxn_id = node_id
            if '@' in rxn_id:
                rxn_id, database = rxn_id.split('@')
            #print(rxn_id, rxn_cmps, len(cmp_to_rxn[rxn_cmps]))
            if rxn_id in cmp_to_rxn[rxn_cmps]:
                remap[(node_id, rxn_cmps)] = cmp_to_rxn[rxn_cmps][rxn_id]
                mapped.add(node_uid)
                #print('yes', rxn_cmps, cmp_to_rxn[rxn_cmps][rxn_id])
        #print(cstoich_list, rxn_cmps, rnode)
        #break
                #print(m, cmp, cpd_id, cmp_to_cpd[cmp][cpd_id])
        #m['compartment'] = 'c0'
        #m['database'] = 'SEED'
        #print(m)
                #break
        if not node_uid in mapped:
            unmapped.add(node_id)
    return remap, unmapped


In [88]:
em_fungi_master = clone_escher_map(em_seed)
#em_fungi_master = clone_escher_map(em_base_seed)
cpd_seed_to_target, unmapped_cpd = escher_map_to_model_metabolites(em_fungi_master, master_fungi)
rxn_seed_to_target, unmapped_rxn = escher_map_to_model_reactions(em_fungi_master, master_fungi)

ALLOW_MISSING.add('C00080@KEGG')

def swap_ids2(em, cpd_remap, rxn_remap):
    swap_node = {}
    for map_uid in em.escher_graph['nodes']:
        node = em.escher_graph['nodes'][map_uid]
        cmp = node['compartment'] if 'compartment' in node else None
        #print(cmp, node['bigg_id'], (node['bigg_id'], cmp) in cpd_remap)
        
        if node['node_type'] == 'metabolite' and (node['bigg_id'], cmp) in cpd_remap:
            #print(node['bigg_id'], cmp, cpd_remap[(node['bigg_id'], cmp)])
            node['bigg_id'] = cpd_remap[(node['bigg_id'], cmp)]
            swap_node[map_uid] = node['bigg_id']
            
        #    node['bigg_id'] = cpd_remap[node['bigg_id']]
    unmapped_rxn = set()
    for map_uid in em.escher_graph['reactions']:
        map_reaction = em.escher_graph['reactions'][map_uid]
        cstoich_list = get_cstoich_list(em, map_reaction)
        rxn_cmps = tuple(sorted(set(map(lambda x : x[1] + '0', cstoich_list))))
        #print(map_uid, rxn_cmps)
        if (map_reaction['bigg_id'], rxn_cmps) in rxn_remap:
            missing = set()
            
        #    map_reaction['bigg_id'] = rxn_remap[map_reaction['bigg_id']]
            for m in map_reaction['metabolites']:
                if not m['node_uid'] in swap_node:
                    missing.add(m['node_uid'])
                    #print(m, swap_node[m['node_uid']])
                #else:
                #    print('?', m)
            if len(missing) == 0:
                #print('yes!', rxn_remap[(map_reaction['bigg_id'], rxn_cmps)])
                map_reaction['bigg_id'] = rxn_remap[(map_reaction['bigg_id'], rxn_cmps)]
                for m in map_reaction['metabolites']:
                    m['bigg_id'] = swap_node[m['node_uid']]
            else:
                unmapped_rxn.add(map_reaction['bigg_id'])
        #    if m['bigg_id'] in cpd_remap:
        #        m['bigg_id'] = cpd_remap[m['bigg_id']]
    return unmapped_rxn
                
swap_ids2(em_fungi_master, cpd_seed_to_target, rxn_seed_to_target)
em_fungi_master.delete_reactions(unmapped_rxn)
b = escher.Builder(map_json=json.dumps(em_fungi_master.escher_map), model_json=json.dumps(master_fungi))
b.set_highlight_missing(True)
b.display_in_notebook(enable_editing=False)
b.save_html('../../../data/www/fungi_viz/master.html', overwrite=True)

'../../../data/www/fungi_viz/master.html'

In [89]:
def model_remap_map_compounds(base_model, em, annotation, bigg_to_seed, id_function = lambda x : (x, 0), 
                              em_id_function = lambda x : x):
    map_compound_remap = {}
    unmaped = set()
    node_uid_cmp = {}
    
    model_metabolites = {}
    
    for m in base_model['metabolites']:
        #print(m)
        if 'annotation' in m and annotation in m['annotation']:
            model_metabolites[m['id']] = m['annotation'][annotation]
            
    #print(model_metabolites)
    
    for map_uid in em.escher_graph['nodes']:
        node = em.escher_graph['nodes'][map_uid]
        if node['node_type'] == 'metabolite':
            node_id = node['bigg_id']
            bigg_id, cmp = id_function(node_id)
            #bigg_id = node_id[:-2]
            #cmp = node_id[-1:]
            
            #print(node_id, bigg_id, cmp)
            if bigg_id in model_metabolites:
                node_uid_cmp[map_uid] = cmp
                map_compound_remap[node_id] = set(map(em_id_function, model_metabolites[bigg_id]))
                logger.debug("bigg_id[%s] cmp[%s] -> [%s]", bigg_id, cmp, map_compound_remap[node_id])
                #print(map_uid, bigg_id, cmp, bigg_to_seed[bigg_id])
            else:
                logger.debug("bigg_id[%s] cmp[%s] NO MAPPING", bigg_id, cmp)
                unmaped.add(node_id)
                
            
    return map_compound_remap, unmaped, node_uid_cmp
em_annotation = clone_escher_map(em_fungi_master)
map_compound_remap, unmaped_cpd, node_uid_cmp = model_remap_map_compounds(master_fungi, em_annotation, 
                                                                              model_id, {}, 
                                                                              id_function = lambda x : (x, x[-2:]),
                                                                              em_id_function = lambda x : x.replace('-45-', '__45__'))


In [90]:


def model_remap_map_reactions(base_model, em, annotation, bigg_to_seed_rxn, map_compound_remap, get_rxn, to_match_func = lambda x : x):
    unmaped_rxn = set()
    map_reaction_remap = {}
    
    model_reactions = {}
    
    for r in base_model['reactions']:
        if 'annotation' in r and annotation in r['annotation']:
            model_reactions[r['id']] = r['annotation'][annotation]

    for map_uid in em.escher_graph['reactions']:
        rnode = em.escher_graph['reactions'][map_uid]
        node_id = rnode['bigg_id']
        if node_id in model_reactions:
            for db_id in model_reactions[node_id]:

                map_stoichiometry = get_stoichiometry(rnode)
                
                logger.debug("%s %s %s", node_id, db_id, map_stoichiometry)
                #print(db_id)
                #NEED KEGG/METACYC/BIGG provenance
                
                rxn_cstoich = get_rxn(db_id)
                logger.debug("%s %s", db_id, rxn_cstoich)

                #print(rxn_cstoich)
                if rxn_cstoich != None:
                    to_match = set(map(lambda x : to_match_func(x[0]), rxn_cstoich.keys()))
                    logger.debug("Match: %s -> %s", to_match, map_stoichiometry)
                    #print(to_match, map_stoichiometry)
                    mapping, missing, to_match = match2(to_match, map_stoichiometry, map_compound_remap)
                    logger.debug("mapping/missing: %s %s", mapping, missing)
                    missing -= ALLOW_MISSING
                    #print(missing)
                    if len(missing) == 0:
                        #print(map_uid, node_id, db_id, map_stoichiometry, rxn.cstoichiometry)
                        #print(mapping, missing, to_match)
                        if not node_id in map_reaction_remap:
                            map_reaction_remap[node_id] = set()
                        map_reaction_remap[node_id].add(db_id)
                        for s_id in rnode['segments']:
                            s = rnode['segments'][s_id]
                            #print(s_id, s)
                            break
                    else:
                        logger.debug("node_id, db_id, missing: %s %s %s", node_id, db_id, missing)
                        print(node_id, "?", missing)
                        unmaped_rxn.add(node_id)
                else:
                    logger.debug('Original Stoich not found: %s [node:%s]', db_id, node_id)
                    unmaped_rxn.add(node_id)
        else:
            logger.debug('Not mapped: [node:%s]', node_id)
            unmaped_rxn.add(node_id)
    return map_reaction_remap, unmaped_rxn

def get_raw_stoich2(db_id, target_db):
    cobra_rxn = None
    if target_db in models_raw_cobra:
        cobra_model = models_raw_cobra[target_db]
        try:
            cobra_rxn = cobra_model.reactions.get_by_id(db_id)
        except KeyError:
            try:
                if db_id[:2] == "R_":
                    cobra_rxn = cobra_model.reactions.get_by_id(db_id[2:])
            except KeyError as e:
                return None
    
    if not cobra_rxn == None:
        cstoich = {}
        for m in cobra_rxn.metabolites:
            cstoich[(m.id, m.compartment)] = cobra_rxn.metabolites[m]
        return cstoich
    return None

def add_namespace2(d, ns_key, ns_value):
    res = {}
    for k in d:
        k_ns = k if ns_key == None else k + '@' + ns_key
        res[k_ns] = set()
        res[k_ns] = d[k] + '@' + ns_value
    return res

In [91]:
def get_next_id(em):
    next_id = 0
    for node_id in em.escher_graph['nodes']:
        if int(node_id) >= next_id:
            next_id = int(node_id) + 1

    return next_id

def translate_metabolite_node(n, lookup, x_off = 0, y_off = 0):
    #print(n)
    bigg_id, name = lookup(n)
    #print(bigg_id, name)
    #id = node['bigg_id']
    #id = id[:-2]
    if bigg_id == None:
        raise Exception('not found ' + n['bigg_id'])
    
    result = copy.deepcopy(n)
    result['x'] += x_off
    result['y'] += y_off
    result['label_x'] += x_off
    result['label_y'] += y_off
    result['bigg_id'] = bigg_id
    result['name'] = name
    #result = {
    #    'x': n['x'] + x_off, 
    #    'y': n['y'] + y_off, 
    #    'name': name, 
    #    'node_is_primary': n['node_is_primary'], 
    #    'node_type': 'metabolite', 
    #    'label_x': n['label_x'] + x_off, 
    #    'label_y': n['label_y'] + y_off,
    #    'bigg_id': bigg_id
    #}
    return result

def translate_b(segment, x_off, y_off):
    if segment == None:
        return None
    else:
        return {
            'x' : segment['x'] + x_off, 
            'y' : segment['y'] + y_off}

def translate_reaction_node(rnode, lookup, next_id, x_off = 0, y_off = 0,
                            node_id_remap = {}, compound_remap = {}, allow_miss = True):
    #print(compound_remap)
    id = rnode['bigg_id']
    bigg_id, name = lookup(rnode)
    if bigg_id == None:
        if allow_miss:
            bigg_id = rnode['bigg_id']
            name = rnode['name']
        else:
            raise Exception('not found ' + rnode['bigg_id'])

    metabolites = []
    segments = {}
    for m in rnode['metabolites']:
        cpd_id = compound_remap[m['bigg_id']] if m['bigg_id'] in compound_remap else '?'
        metabolites.append({
            'bigg_id' : cpd_id,
            'coefficient' : m['coefficient']
        })
    for s in rnode['segments']:
        segment = rnode['segments'][s]
        if segment['from_node_id'] in node_id_remap and segment['to_node_id'] in node_id_remap:
            segment_translate = {
                'b1' : translate_b(segment['b1'], x_off, y_off),
                'b2' : translate_b(segment['b2'], x_off, y_off),
                'from_node_id' : node_id_remap[segment['from_node_id']],
                'to_node_id' : node_id_remap[segment['to_node_id']]
            }
            segments[str(next_id)] = segment_translate
            next_id += 1
    result = {
        'bigg_id': bigg_id,
        'name': name,
        'gene_reaction_rule': '',
        'genes': [],
        'label_x': rnode['label_x'] + x_off,
        'label_y': rnode['label_y'] + y_off,
        'metabolites': metabolites,
        'reversibility': rnode['reversibility'],
        'segments': segments
    }
    return result, next_id

def copy_map_compounds(escher_map, escher_map_target, next_id, cpd_lookup, x_off = 0, y_off = 0):
    compound_remap = {}
    node_id_remap = {}
    id_counter = next_id
    for node_id in escher_map[1]['nodes']:
        node = escher_map[1]['nodes'][node_id]
        note_type = node['node_type']
        if note_type == 'metabolite':
            #print(node)
            tnode = translate_metabolite_node(node, cpd_lookup, x_off, y_off)
            #print(node, tnode)
            #print(node['bigg_id'], tnode['bigg_id'])
            compound_remap[node['bigg_id']] = tnode['bigg_id']
            escher_map_target[1]['nodes'][str(id_counter)] = tnode
        else:
            escher_map_target[1]['nodes'][str(id_counter)] = {
                'x': node['x'] + x_off,
                'y': node['y'] + y_off,
                'node_type': node['node_type']
            }
        node_id_remap[node_id] = str(id_counter)
        id_counter += 1
    
    return id_counter, node_id_remap, compound_remap

def copy_map_reactions(escher_map, escher_map_target, next_id, node_id_remap, compound_remap, rxn_lookup, x_off = 0, y_off = 0):
    id_counter = next_id
    for rnode_id in escher_map[1]['reactions']:
        rnode = escher_map[1]['reactions'][rnode_id]

        trnode, id_counter = translate_reaction_node(
            rnode, rxn_lookup, id_counter, 
                                                  x_off, y_off, 
                                                  node_id_remap, compound_remap)

        escher_map_target[1]['reactions'][str(id_counter)] = trnode
        id_counter += 1
    
    return id_counter

def grid_maps(maps, grid):
    master = clone_escher_map(maps[0])
    block_width = master.escher_graph['canvas']['width']
    block_height = master.escher_graph['canvas']['height']
    master.escher_graph['canvas']['width'] *= grid[0]
    master.escher_graph['canvas']['height'] *= grid[1]
    next_id = get_next_id(maps[0])
    print(next_id)
    map_index = 1
    for i in range(0, grid[1]):
        for j in range(0, grid[0]):
            logger.debug("[%d, %d]", i, j)
            if not (i == 0 and j == 0) and map_index < len(maps):
                if not maps[map_index] == None:
                    logger.debug("[DRAW] %d next_id: %d", map_index, next_id)
                    
                    next_id, node_id_remap, compound_remap = copy_map_compounds(maps[map_index].escher_map,
                                       master.escher_map, 
                                       next_id, 
                                       lambda n : (n['bigg_id'], n['name']), 
                                       x_off = j * block_width, y_off = i * block_height)
                    
                    next_id = copy_map_reactions(maps[map_index].escher_map, 
                                                 master.escher_map, 
                                                 next_id, 
                                                 node_id_remap, compound_remap, 
                                                 lambda n : (n['bigg_id'], n['name']), 
                                                 x_off = j * block_width, y_off = i * block_height)
                map_index += 1
        
    return master

In [92]:
%%capture

models_raw_json_path = {
    'iMM904' : '../../../data/www/fungi_viz/data/sbml/iMM904.json',
    #'iJL1454_KBase2' : '../../../data/www/fungi_viz/data/sbml/iJL1454.xml',
    'iAL1006' : '../../../data/www/fungi_viz/data/sbml/iAl1006.json',
    #'iLC915_KBase2' : '../../../data/www/fungi_viz/data/sbml/iLC915.json',
    #'iMA871_KBase2' : '../../../data/www/fungi_viz/data/sbml/iMA871.json',
    #'iNL895_KBase2' : '../../../data/www/fungi_viz/data/sbml/iNL895.json',
    #'iCT646_KBase2' : '../../../data/www/fungi_viz/data/sbml/i',
    'iJDZ836' : '../../../data/www/fungi_viz/data/sbml/iJDZ836.json',
    #'iRL766_KBase2' : '../../../data/www/fungi_viz/data/sbml/iRL766.json',
    #'iSS884_KBase2' : '../../../data/www/fungi_viz/data/sbml/iSS884.json',
    #'iCT646_KBase2' : '../../../data/www/fungi_viz/data/sbml/iCT646.json',
    #'yeast_6.06_KBase2' : '../../../data/www/fungi_viz/data/sbml/yeast_6.06.json',
    #'yeast_7.6_KBase2' : '../../../data/www/fungi_viz/data/sbml/yeast_7.6.json',
}

models_raw_cobra = {}
for k in models_raw_json_path:
    #models_raw_cobra[k] = cobra.io.read_sbml_model(models_raw_sbml_path[k])
    models_raw_cobra[k] = cobra.io.from_json(json.dumps(read_json(models_raw_json_path[k])))
    #models_raw_bios[k] = read_json(models_raw_json_path[k])

In [93]:
def tag_model(cobra_json, model_tag):
    res = copy.deepcopy(cobra_json)
    for o in res['metabolites']:
        o['id'] += '@' + model_tag
    for o in res['reactions']:
        o['id'] += '@' + model_tag
        m_tag = {}
        for m in o['metabolites']:
            m_tag[m + '@' + model_tag] = o['metabolites'][m]
        o['metabolites'] = m_tag

    return res

merge_tag_model = None
for model_id in models:
    model_tag = models_tags[model_id] if model_id in models_tags else model_id
    print(model_id, model_tag)
    cobra_json = json.loads(cobra.io.to_json(models_raw_cobra[model_id]))
    if merge_tag_model == None:
        merge_tag_model = tag_model(cobra_json, model_tag)
    else:
        tmodel = tag_model(cobra_json, model_tag)
        for o in tmodel['metabolites']:
            merge_tag_model['metabolites'].append(o)
        for o in tmodel['reactions']:
            merge_tag_model['reactions'].append(o)

iMM904 iMM904
iAL1006 iAL1006
iJDZ836 iJDZ836


In [94]:
%%capture
logger.setLevel(logging.WARNING)

model_get_rxn = {
    #'iJDZ836_KBase2' : lambda x : get_raw_stoich2(x.replace('-', '__'), model_id)
    #'iJDZ836_KBase2' : jdz_get_rxn
}

remap_compound_identifiers = {
    #'iMM904_KBase2' : lambda d : dict(map(lambda x : (x[0], set(map(bigg1_to_bigg2, x[1]))), d.items())),
    'iNL895_KBase2' : lambda d : dict(map(lambda x : (x[0], set(map(strip_M, x[1]))), d.items())),
    #'iSS884_KBase2' : lambda d : dict(map(lambda x : (x[0], set(map(strip_M, x[1]))), d.items())),
}

post_rxn_id_func = {
    'iJDZ836_KBase2' : lambda d : dict(map(lambda x : (x[0], set(map(strip_Rspecial, map(jdz_revert, x[1])))), d.items()))
}

em_models = {}
models = [
    'iMM904',
    'iAL1006',
    'iJDZ836'
]
#models = models_raw_cobra.keys()

models_tags = {
    'iMM904' : 'iMM904',
    #'iJL1454_KBase2' : '../../../data/www/fungi_viz/data/sbml/iJL1454.xml',
    'iAL1006' : 'iAL1006',
    'iLC915_KBase2' : 'iLC915',
    'iMA871_KBase2' : 'iMA871',
    'iNL895_KBase2' : 'iNL895',
    #'iCT646_KBase2' : '../../../data/www/fungi_viz/data/sbml/i',
    'iJDZ836' : 'iJDZ836',
    'iSS884_KBase2' : 'iSS884',
    'iRL766_KBase2' : 'iRL766',
    
    'yeast_6.06_KBase2' : 'yeast_6.06',
    'yeast_7.6_KBase2' : 'yeast_7.6',
    'iCT646_KBase2' : 'iCT646',
}

for model_id in models:
    #print(model_id)
    em_annotation = clone_escher_map(em_fungi_master)
    get_rxn_func = lambda x : get_raw_stoich2(x, model_id)
    if model_id in model_get_rxn:
        get_rxn_func = model_get_rxn[model_id]
    map_compound_remap, unmaped_cpd, node_uid_cmp = model_remap_map_compounds(master_fungi, em_annotation, 
                                                                              model_id, {}, 
                                                                              id_function = lambda x : (x, x[-2:]),
                                                                              em_id_function = lambda x : x.replace('-45-', '__45__'))
    #print(map_compound_remap)
    if model_id in remap_compound_identifiers:
        map_compound_remap = remap_compound_identifiers[model_id](map_compound_remap)
    
    map_reaction_remap, unmaped_rxn = model_remap_map_reactions(master_fungi, em_annotation, 
                                                                model_id, {}, map_compound_remap, get_rxn_func)
    #print(map_reaction_remap)
    for k in map_compound_remap:
        if len(map_compound_remap[k]) > 1:
            print('***', k, map_compound_remap[k])
    em_annotation.delete_reactions(unmaped_rxn)
    em_annotation.delete_metabolites(unmaped_cpd)
    
    if model_id in post_rxn_id_func:
        map_reaction_remap = post_rxn_id_func[model_id](map_reaction_remap)
    
    cpd_remap = {}
    rxn_remap = {}
    for k in map_compound_remap:
        cpd_remap[k] = list(map_compound_remap[k])[0]
    for k in map_reaction_remap:
        rxn_remap[k] = list(map_reaction_remap[k])[0]
    model_tag = models_tags[model_id] if model_id in models_tags else model_id
    cpd_remap = add_namespace2(cpd_remap, None, model_tag)
    rxn_remap = add_namespace2(rxn_remap, None, model_tag)
    em_annotation.swap_ids(cpd_remap, rxn_remap)
    em_models[model_id] = em_annotation

In [95]:
em_maps = [em_seed, em_fungi_master, None]
for k in em_models:
    em_maps.append(em_models[k])
master = grid_maps(em_maps, (3, 3))
#cobra_json = json.loads(cobra.io.to_json(models_raw_cobra['iMM904_KBase2']))
#cobra_json = tag_model(cobra_json, 'iMM904')
b = escher.Builder(map_json=json.dumps(master.escher_map), model_json=json.dumps(merge_tag_model))
b.set_highlight_missing(True)
b.display_in_notebook()

681711


In [96]:
b.save_html('../../../data/www/fungi_viz/debug.html', overwrite=True)

'../../../data/www/fungi_viz/debug.html'

In [31]:
master_fungi = read_json('../../notebooks/fungi_map/fungi_viz/data/master_model3.json')
merge_model = cobra.io.from_json(json.dumps(master_fungi))


In [32]:
for r in merge_model.reactions:
    print(r)
    break

rxn01256_c0: cpd00216_c0 --> cpd00219_c0


In [36]:
rxn = merge_model.reactions.get_by_id('rxn00799_m0')
rxn.annotation

{'iJDZ836_KBase3': ['FUMHYDR__45__RXN__45__L2R__91__CCO__45__MIT__93__']}